In [25]:
%load_ext autoreload
%autoreload 2

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=5000, n_features=10, n_informative=2, n_redundant=0)
# X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(10)])

## One by one test

In [40]:
import sys
sys.path.append("../powershap")

from powershap import PowerSHAP


from catboost import CatBoostClassifier

selector = PowerSHAP(
    model = CatBoostClassifier(verbose=0, n_estimators=200),
    power_iterations=10,
    # automatic=True,
    # limit_automatic=100,
)

In [41]:
selector.fit(X, y)

100%|██████████| 10/10 [00:14<00:00,  1.43s/it]

Failed to converge on a solution.

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


PowerSHAP(model=<catboost.core.CatBoostClassifier object at 0x7f3b32b342b0>)

In [42]:
selector.transform(X)

array([[ 1.05177032,  0.20433431, -0.71197018],
       [ 0.63657333, -0.46774804,  1.71724841],
       [-0.07523655,  0.49548664,  2.17273377],
       ...,
       [-1.56064525,  0.90378236,  2.5407778 ],
       [-0.80927141, -0.56775875, -0.85677633],
       [ 1.12766922,  1.20293023,  1.0179223 ]])

In [43]:
selector._processed_shaps_df

,impact,p_value,effect_size,power_0.01_alpha,0.95_power_its_req
4,3.832704,0.0,75.666994,1.0,[10.0]
1,1.293248,0.0,25.108043,1.0,2.180626
2,0.15392,0.0,2.006632,0.998462,6.911325
8,0.09663,0.5,0.397909,0.106235,102.335507
0,0.089965,0.6,0.097972,0.019747,1645.717178
random_uniform_feature,0.087199,0.6,0.0,0.01,0
5,0.082373,0.6,0.193992,0.036092,421.771006
6,0.077554,0.8,0.402957,0.108748,99.856611
7,0.075366,0.7,0.583175,0.226684,49.130233
3,0.073204,0.7,0.689173,0.320232,35.982082


## sklearn pipeline test

In [44]:
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression

pipe = Pipeline(
    [
        (
            "selector",
            PowerSHAP(
                CatBoostClassifier(
                    verbose=0, n_estimators=250,
                ), automatic=True, limit_automatic=1000,
            ),
        ),
        ("lr", LogisticRegression()),
    ]
)

pipe.fit(X, y)


from sklearn.metrics import accuracy_score

accuracy_score(pipe.predict(X), y)


100%|██████████| 10/10 [00:13<00:00,  1.32s/it]


0.9116